In [1]:
from transformers import AutoModel,AutoTokenizer,pipeline,Trainer,TrainingArguments,AutoModelForCausalLM,default_data_collator
from datasets import Dataset, load_dataset
import numpy as np
import evaluate

[2025-06-13 19:21:21,779] [INFO] [real_accelerator.py:254:get_accelerator] Setting ds_accelerator to cuda (auto detect)


W0613 19:21:23.055000 8720 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


[2025-06-13 19:21:23,514] [INFO] [logging.py:107:log_dist] [Rank -1] [TorchCheckpointEngine] Initialized with serialization = False


### 加载模型和分词器

In [2]:
model_name="Qwen/Qwen3-0.6B"

In [3]:
model=AutoModelForCausalLM.from_pretrained(
    model_name,
    # trust_remote_code=True,
    device_map='auto',
    torch_dtype="auto",
)
tokenizer=AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token=tokenizer.eos_token

### 加载训练数据

In [4]:
dataset = load_dataset('json', data_files={
    'train': '../数据集/data/train.json',
    'validation': '../数据集/data/eval.json'
})

In [5]:
print(f"数据集加载成功，包含以下部分: {list(dataset.keys())}")
print(f"训练数据集加载成功，包含 {len(dataset['train'])} 个样本")
print(f"验证数据集加载成功，包含 {len(dataset['validation'])} 个样本")

数据集加载成功，包含以下部分: ['train', 'validation']
训练数据集加载成功，包含 320 个样本
验证数据集加载成功，包含 30 个样本


In [6]:
print(f"训练数据集示例: {dataset['train'][0]}")

训练数据集示例: {'conversations': [{'from': 'human', 'value': '请问室温超导技术的难点在哪里,人类在什么时候可以实现室温超导?'}, {'from': 'gpt', 'value': '室温超导技术的难点在于目前尚未发现或创造出能在室温下保持超导性的材料。常见的超导体需要在极低温（如液氦温度，约-269°C）下才能表现出零电阻和完全抗磁性的超导特性。尽管科学家们在高温超导材料研究方面取得了进展，目前已知的高温超导体在实现超导状态时所需的温度仍远远低于室温。\n\n实现室温超导的难点主要有以下几个方面：\n\n1. 材料问题：高温超导体多为陶瓷材料，这类材料往往脆弱难以制成长导线，并且许多超导材料在高温、高压条件下稳定性较差。\n\n2. 理论挑战：超导机制通常基于量子物理理论。对于传统的低温超导体，BCS理论（巴丁-库珀-施里弗理论）能够很好地描述其超导机制。但对于高温超导体，至今还没有一个普遍认可且完整的理论框架。\n\n3. 技术限制：即使找到合适的材料或者理论模型，通过现有的技术制备高质量的样品也是非常困难的。要保持超导状态需要的一系列条件如纯度、晶格结构的完整性等，在室温下实现和维持更加困难。\n\n4. 经济和实用性：即便科学家们能发现或制造出在室温下工作的超导材料，但其制造成本、可持续性和实际应用的有效性都是可能的挑战。\n\n至于人类能在什么时候实现室温超导，目前无法给出一个确切的时间表。室温超导是物理学中的一个激动人心但又极具挑战性的前沿议题，其研究进展依赖于新材料的发现、理论物理的突破以及实验技术的进步。可以说，室温超导的实现还处于科学探索的阶段，是未来技术发展具有重要潜力的领域之一。'}, {'from': 'human', 'value': '可以列出5样稀有金属吗？请按全球需求来排名'}, {'from': 'gpt', 'value': '稀有金属指的是地壳中含量较少、分布不均或开采困难的金属元素，它们在高科技领域和新兴工业中扮演着非常重要的角色。全球需求的排序可能因时间和技术进步而变化，但以下是通常被广泛需求的一些稀有金属（不一定按绝对的需求量排名，因为这在不同时间点可能不同）：\n\n1. **钴(Cobalt, Co)** - 钴是锂离子电池的关键成分之一，尤其是在电动汽车和便携式电子产

In [7]:
print(f"验证数据集示例: {dataset['validation'][0]}")

验证数据集示例: {'conversations': [{'from': 'human', 'value': '我想系统的学习编程，该如何开始？'}, {'from': 'gpt', 'value': '系统地学习编程非常重要，以下是一些步骤和建议帮助你开始：\n\n### 1. 选择编程语言\n开始学习编程之前，你需要选择一门编程语言。对于初学者来说，常推荐的语言包括Python、JavaScript、Ruby或Java。Python由于其简洁的语法和强大的社区支持，通常是入门的首选。\n\n### 2. 理解编程基础\n不论选择哪种语言，都应从基础概念开始学习，如：\n- 数据类型和变量\n- 控制结构（条件语句、循环）\n- 数据结构（数组、列表、字典）\n- 函数/方法\n- 类和对象（面向对象编程的基本）\n\n### 3. 搭建开发环境\n- 为你选择的语言安装编译器或解释器。\n- 安装文本编辑器或集成开发环境（IDE），例如Visual Studio Code、PyCharm或Eclipse。\n\n### 4. 实践编码\n- 开始写简单的程序，实践你所学的概念。\n- 尝试解决一些基本的编程问题，例如LeetCode、HackerRank或Codecademy上的挑战。\n\n### 5. 学习版本控制\n- 学习使用Git，并熟悉GitHub或其他代码托管平台，以便管理你的代码版本和参与开源项目。\n\n### 6. 开始小项目\n- 一边学习，一边开始一个小项目，比如一个个人博客、一个简单的游戏或者一个数据分析的小工具。\n- 实际动手解决问题能增强你的学习经验和解决问题的能力。\n\n### 7. 学习计算机科学原理\n- 学习计算机科学的基础，如算法和数据结构，这会帮助你写出更高效、可维护的代码。\n- 了解操作系统、网络和数据库的基础知识也非常重要。\n\n### 8. 参考优质资源\n- 书籍：找一些好评如潮的编程书籍开始阅读，如《Python编程：从入门到实践》、《JavaScript高级程序设计》等。\n- 在线课程：Udemy、Coursera、edX 提供了非常多的在线课程。\n- 视频教程：YouTube、B站等平台有很多免费的编程教程。\n\n### 9. 加入社区\n- 加入Stack Overflow、Reddit、编

### 数据预处理

In [8]:
def process_fun(example):
    question=[]
    answer=[]
    for i in example['conversations']:
        for j in i:
            if j['from']=='human':
                question.append(j['value'])
            elif j['from']=='gpt':
                answer.append(j['value'])
    return {'question':question,'answer':answer}

In [9]:
process_data=dataset.map(process_fun,batched=True,remove_columns=dataset['train'].column_names) # 提取问题和答案数据预处理

In [10]:
def tokenizer_fun(examples):
    # 构建完整的指令格式（问：{问题}\n答：{答案}）
    instructions = []
    for q, a in zip(examples['question'], examples['answer']):
        instruction = f"问：{q}\n答：{a}"
        instructions.append(instruction)
    
    encoded = tokenizer(
        instructions,
        max_length=512,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )
    
    labels = encoded["input_ids"].clone()
    
    # 定位"答："的位置，标记需要预测的部分
    answer_start_token = tokenizer.encode("答：", add_special_tokens=False)[0]
    
    # 遍历批次中的每个样本
    for i in range(len(labels)):
        # 找到每个样本中"答："的第一个token位置
        answer_positions = (labels[i] == answer_start_token).nonzero(as_tuple=True)[0]
        if len(answer_positions) > 0:
            # 只取第一个"答："的位置
            first_answer_pos = answer_positions[0]
            # 将"答："之前的token标记为-100（忽略计算损失）
            labels[i, :first_answer_pos] = -100
    
    return {
        "input_ids": encoded["input_ids"],
        "attention_mask": encoded["attention_mask"],
        "labels": labels
    }

In [11]:
tokenized_dataset = process_data.map(
    tokenizer_fun,
    batched=True,
    remove_columns=process_data['train'].column_names
)

In [12]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 606
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 52
    })
})

### 训练超参数配置

In [13]:
# 定义训练参数
training_args = TrainingArguments(
    output_dir="./train_qwen_0.6B_model",
    logging_steps=100,
    logging_dir='./runs',
    eval_strategy='epoch',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=2e-5,
    weight_decay=0.01,
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
    gradient_accumulation_steps=4,  # 如果GPU内存有限
)

### 训练模型

In [14]:
trainer=Trainer(
    model=model,
    args=training_args,
    eval_dataset=tokenized_dataset["validation"],
    train_dataset=tokenized_dataset["train"],
    data_collator=default_data_collator,
)

In [15]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,1.402303
2,No log,1.392737
3,1.439400,1.391980


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=114, training_loss=1.4139219585217928, metrics={'train_runtime': 4291.1831, 'train_samples_per_second': 0.424, 'train_steps_per_second': 0.027, 'total_flos': 2459964933144576.0, 'train_loss': 1.4139219585217928, 'epoch': 3.0})

In [16]:
#手动保存模型
trainer.save_model('./train_qwen')
trainer.save_state()